**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**3) Configure Darknet network for training YOLO V3**

**NOTE:**

FILTER = (c+5)*3

max_batches = c*2000 (unless it's smaller than 6000)

steps = %80, %90*max_batches


In [ ]:
!cp cfg/yolov4-csp.cfg cfg/yolov4-training.cfg

In [ ]:
!sed -i 's/batch=64/batch=64/' cfg/yolov4-training.cfg
!sed -i 's/subdivisions=8/subdivisions=32/' cfg/yolov4-training.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov4-training.cfg
!sed -i 's/steps=400000,450000/steps=4800,5400/' cfg/yolov4-training.cfg
!sed -i '1034 s@classes=80@classes=1@' cfg/yolov4-training.cfg
!sed -i '1148 s@classes=80@classes=1@' cfg/yolov4-training.cfg
!sed -i '1262 s@classes=80@classes=1@' cfg/yolov4-training.cfg
!sed -i '1027 s@filters=255@filters=18@' cfg/yolov4-training.cfg
!sed -i '1141 s@filters=255@filters=18@' cfg/yolov4-training.cfg
!sed -i '1255 s@filters=255@filters=18@' cfg/yolov4-training.cfg
!sed -i '8 s@width=512@width=640@' cfg/yolov4-training.cfg
!sed -i '9 s@height=512@height=640@' cfg/yolov4-training.cfg

In [ ]:
!printf "person\n" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nnames = data/obj.names\nbackup = /content/drive/MyDrive/weights/person_test/person_test_640' > data/obj.data
!mkdir data/obj

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-csp.conv.142

In [ ]:
!mkdir /content/images/

import glob, os
zip_list = glob.glob("/content/drive/MyDrive/dataset/person_test/*.zip")
rar_list = glob.glob("/content/drive/MyDrive/dataset/person_test/*.rar")

for zip in zip_list:
  print(zip, " zip")
  os.system("unzip {} -d /content/images/".format(zip))

for rar in rar_list:
  print(rar, " rar")
  os.system("unrar e {} /content/images/".format(rar))

In [ ]:
import glob
images_list = glob.glob("/content/images/*.jpg")
for txt in glob.glob("/content/images/*.txt"):
  read = []
  write = []
  f = open("{}".format(txt), "r+")
  read = f.readlines()
  for i in range(len(read)):
      a = read[i]
      another_l = a.split(" ")
      if another_l[0] != "0":
          another_l[0] = "0"
      write.append(another_l)
  f = open("{}".format(txt), "w")
  for i in range(len(write)):
      for j in range(len(write[i])):
          if write[i][j] == write[i][-1]:
              pass
          else:
              write[i][j] += " "
      f.writelines(write[i])
  f.close()
  

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 



**6) Start the training**

darknet53.conv.74

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov4-training.cfg /content/drive/MyDrive/weights/person_test/person_test_640/last.weights -dont_show